## Leitura do arquivo

In [32]:
with open('dados/artigos.txt', 'r') as f:
  artigos = f.read()

##Tokenização

In [33]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
def separa_palavras(lista_tokens):
  lista_palavras = []
  for token in lista_tokens:
    if token.isalpha():
      lista_palavras.append(token)
  return lista_palavras

In [35]:
lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_palavras = separa_palavras(lista_tokens)
print('O número de palavras no corpus é {}.'.format(len(lista_palavras)))

O número de palavras no corpus é 403031.


In [36]:
def normalizacao(lista_palavras):
  lista_normalizada = []
  for palavra in lista_palavras:
    lista_normalizada.append(palavra.lower())
  return lista_normalizada

In [37]:
lista_normalizada = normalizacao(lista_palavras)
print('O número de palavras que o modelo vai aprender: {}.'.format(len(set(lista_normalizada))))

O número de palavras que o modelo vai aprender: 18464.


# Inserção de letras na palavra

In [42]:
def gerador_palavras(palavra):
  fatias = []
  for i in range(len(palavra)+1):
    fatias.append((palavra[:i], palavra[i:]))
  palavras_geradas = insere_letras(fatias)
  return palavras_geradas

def insere_letras(fatias):
  novas_palavras = []
  letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
  for e, d in fatias:
    for letra in letras:
      novas_palavras.append(e + letra + d)
  return novas_palavras

def corretor(palavra):
  palavras_geradas = gerador_palavras(palavra)
  palavra_correta = max(palavras_geradas, key=probabilidade)
  return palavra_correta

def probabilidade(palavras_geradas):
  frequencia = nltk.FreqDist(lista_normalizada)
  return frequencia[palavras_geradas]/total_palavras

# Teste

In [40]:
total_palavras = len(lista_normalizada)

In [41]:
palavra_exemplo = 'lgica'
corretor(palavra_exemplo)

'lógica'